In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_input = pd.read_csv(r'C:\DATASETS\day5\2019-Dec.csv')
df_input.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [3]:
df_target = pd.read_csv(r'C:\DATASETS\day5\2020-Jan.csv')
df_target.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,5809910,1602943681873052386,NaN,grattol,5.24,595414620,4adb70bb-edbd-4981-b60f-a05bfd32683a
1,2020-01-01 00:00:09 UTC,view,5812943,1487580012121948301,NaN,kinetics,3.97,595414640,c8c5205d-be43-4f1d-aa56-4828b8151c8a
2,2020-01-01 00:00:19 UTC,view,5798924,1783999068867920626,NaN,zinger,3.97,595412617,46a5010f-bd69-4fbe-a00d-bb17aa7b46f3
3,2020-01-01 00:00:24 UTC,view,5793052,1487580005754995573,NaN,NaN,4.92,420652863,546f6af3-a517-4752-a98b-80c4c5860711
4,2020-01-01 00:00:25 UTC,view,5899926,2115334439910245200,NaN,NaN,3.92,484071203,cff70ddf-529e-4b0c-a4fc-f43a749c0acb


## Exploratory Data Analysis

In [4]:
df_target.shape

(4264752, 9)

In [5]:
df_target.duplicated().sum()

225100

In [6]:
def pipe_data_clean(df_target):
    df_target.drop_duplicates(inplace=True)
    
    return df_target

In [7]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [8]:
df_target.describe()

,product_id,category_id,price,user_id
count,4264752.00000,4264752.00000,4264752.00000,4264752.00000
mean,5492516.08723,1558391115607577088.00000,8.71398,533522952.77282
std,1291503.32928,174920810627758560.00000,19.63959,87814333.99671
min,3752.00000,1487580004807082752.00000,-79.37000,465496.00000
25%,5724662.00000,1487580005671109376.00000,2.35000,495500727.00000
50%,5811708.00000,1487580008347075584.00000,4.06000,566830470.00000
75%,5859408.00000,1487580013522845952.00000,7.14000,598304946.00000
max,5924514.00000,2235524499636224256.00000,327.78000,608822072.00000


In [9]:
df_target.describe(include='object')

,event_time,event_type,category_code,brand,user_session
count,4264752,4264752,74719,2489122,4263438
unique,1811717,4,12,256,965351
top,2020-01-09 09:06:06 UTC,view,appliances.environment.vacuum,runail,3cfdc45c-8985-4c1d-b1e4-baa16a641e64
freq,257,2037608,32104,368018,2458


In [10]:
df_target.nunique()

event_time       1811717
event_type             4
product_id         45484
category_id          482
category_code         12
brand                256
price               2097
user_id           410073
user_session      965351
dtype: int64

In [11]:
df_target['category_code'].unique()

array([nan, 'accessories.bag', 'stationery.cartrige',
       'appliances.environment.vacuum', 'apparel.glove',
       'appliances.personal.hair_cutter', 'furniture.living_room.cabinet',
       'accessories.cosmetic_bag', 'furniture.bathroom.bath',
       'appliances.environment.air_conditioner',
       'furniture.living_room.chair', 'appliances.personal.massager',
       'sport.diving'], dtype=object)

In [12]:
df_target['category_code'].value_counts()

appliances.environment.vacuum             32104
apparel.glove                             13479
stationery.cartrige                       11469
furniture.living_room.cabinet              6114
furniture.bathroom.bath                    5139
accessories.bag                            4076
appliances.personal.hair_cutter            1225
accessories.cosmetic_bag                    809
appliances.personal.massager                169
appliances.environment.air_conditioner      128
furniture.living_room.chair                   5
sport.diving                                  2
Name: category_code, dtype: int64

## Estruturação dos dados target

In [13]:
df_target['event_type'].unique()

array(['view', 'cart', 'remove_from_cart', 'purchase'], dtype=object)

In [14]:
df_target['fl_purchase'] = df_target['event_type'].apply(lambda x: 1 if x == 'purchase' else 0)

In [15]:
df_purchase = df_target.loc[df_target['fl_purchase']==1]

In [16]:
df_purchase = df_purchase.groupby('user_id').agg({'event_time': 'min',
                                   'price': ['sum', 'mean'],
                                   'product_id': 'nunique',
                                   'user_session': 'nunique'})

In [17]:
df_purchase.columns = ['_'.join(col).strip() if col[1] != '' else col[0] for col in df_purchase.columns.values]

In [18]:
df_purchase.head()

,event_time_min,price_sum,price_mean,product_id_nunique,user_session_nunique
user_id,,,,,
10280338,2020-01-12 22:54:37 UTC,18.10000,18.10000,1,1
12936739,2020-01-17 07:51:19 UTC,29.89000,14.94500,2,1
20554973,2020-01-15 16:12:29 UTC,21.81000,3.11571,7,1
23633626,2020-01-28 09:09:10 UTC,22.13000,2.76625,8,1
27443691,2020-01-27 06:52:19 UTC,29.12000,5.82400,5,1


In [19]:
df_target = df_target.drop(columns=['event_time', 'event_type', 'product_id',
                                    'category_id','category_code', 'brand',
                                   'price', 'user_session'])

In [20]:
df_target

,user_id,fl_purchase
0,595414620,0
1,595414640,0
2,595412617,0
3,420652863,0
4,484071203,0
...,...,...
4264747,607092857,0
4264748,607092857,0
4264749,423651741,0
4264750,607092857,0


In [21]:
df_target.duplicated().sum()

3826599

In [22]:
df_target_final = df_target.drop_duplicates().merge(df_purchase, on='user_id', how='left')

In [23]:
df_target_final.head()

,user_id,fl_purchase,event_time_min,price_sum,price_mean,product_id_nunique,user_session_nunique
0,595414620,0,NaN,nan,nan,nan,nan
1,595414640,0,NaN,nan,nan,nan,nan
2,595412617,0,NaN,nan,nan,nan,nan
3,420652863,0,NaN,nan,nan,nan,nan
4,484071203,0,NaN,nan,nan,nan,nan


## Criação da Pipeline

In [24]:
def pipe_df_target(df_target):
    df_target['fl_purchase'] = df_target['event_type'].apply(lambda x: 1 if x == 'purchase' else 0)
    df_purchase = df_target.loc[df_target['fl_purchase']==1]
    df_purchase = df_purchase.groupby('user_id').agg({'event_time': 'min',
                                   'price': ['sum', 'mean'],
                                   'product_id': 'nunique',
                                   'user_session': 'nunique'})
    df_purchase.columns = ['_'.join(col).strip() if col[1] != '' else col[0] for col in df_purchase.columns.values]
    df_target = df_target.drop(columns=['event_time', 'event_type', 'product_id',
                                    'category_id','category_code', 'brand',
                                   'price', 'user_session'])
    df_target_final = df_target.drop_duplicates().merge(df_purchase, on='user_id', how='left')
    
    return df_target_final

## Estruturação dos dados de input

In [25]:
df_input.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27000,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05000,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19000,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79000,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56000,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [26]:
df_events = pd.pivot_table(df_input, index=['user_id'], columns=['event_type'], values=[
    'event_time', 'product_id', 'category_id', 'brand', 'price', 'user_session'],
                           aggfunc={'user_session': 'nunique',
                                   'price': ['sum', 'mean'],
                                    'product_id': 'nunique',
                                    'category_id': 'nunique',
                                    'brand': 'nunique',
                                    'event_time': 'max'
                                   }, fill_value=0)

In [27]:
df_events.head()

brand                                category_id           \
           nunique                                    nunique            
event_type    cart purchase remove_from_cart view        cart purchase   
user_id                                                                  
1180452          0        0                0    1           0        0   
2963072          0        0                0    2           2        0   
4661182          1        0                1    2           1        0   
4891613          0        0                0    0           0        0   
6217356          0        0                0    0           0        0   

                                               event_time           ...  \
                                                      max           ...   
event_type remove_from_cart view                     cart purchase  ...   
user_id                                                             ...   
1180452                   0    1                        0        0  ...   
2963072                   0    4  2019-12-22 13:12:20 UTC        0  ...   
4661182                   2    2  2019-12-22 19:32:29 UTC        0  ...   
4891613                   0    1                        0        0  ...   
6217356                   0    1                        0        0  ...   

                      price           product_id                            \
                        sum              nunique                             
event_type remove_from_cart      view       cart purchase remove_from_cart   
user_id                                                                      
1180452             0.00000  25.40000          0        0                0   
2963072             0.00000 860.96000          5        0                0   
4661182            23.18000  64.61000          2        0                2   
4891613             0.00000   9.32000          0        0                0   
6217356             0.00000  10.63000          0        0                0   

                user_session                                 
                     nunique                                 
event_type view         cart purchase remove_from_cart view  
user_id                                                      
1180452       1            0        0                0    1  
2963072      10            1        0                0    1  
4661182       2            2        0                1    1  
4891613       1            0        0                0    1  
6217356       1            0        0                0    1  

[5 rows x 28 columns]

In [28]:
df_events.columns = ['_'.join(col).strip() if col[1] != '' else col[0] for col in df_events.columns.values]

In [29]:
df_events.head()

,brand_nunique_cart,brand_nunique_purchase,brand_nunique_remove_from_cart,brand_nunique_view,category_id_nunique_cart,category_id_nunique_purchase,category_id_nunique_remove_from_cart,category_id_nunique_view,event_time_max_cart,event_time_max_purchase,...,price_sum_remove_from_cart,price_sum_view,product_id_nunique_cart,product_id_nunique_purchase,product_id_nunique_remove_from_cart,product_id_nunique_view,user_session_nunique_cart,user_session_nunique_purchase,user_session_nunique_remove_from_cart,user_session_nunique_view
user_id,,,,,,,,,,,,,,,,,,,,,
1180452,0,0,0,1,0,0,0,1,0,0,...,0.00000,25.40000,0,0,0,1,0,0,0,1
2963072,0,0,0,2,2,0,0,4,2019-12-22 13:12:20 UTC,0,...,0.00000,860.96000,5,0,0,10,1,0,0,1
4661182,1,0,1,2,1,0,2,2,2019-12-22 19:32:29 UTC,0,...,23.18000,64.61000,2,0,2,2,2,0,1,1
4891613,0,0,0,0,0,0,0,1,0,0,...,0.00000,9.32000,0,0,0,1,0,0,0,1
6217356,0,0,0,0,0,0,0,1,0,0,...,0.00000,10.63000,0,0,0,1,0,0,0,1


In [30]:
df_train = df_target_final.merge(df_events, on='user_id', how='inner')

In [31]:
df_train.head()

,user_id,fl_purchase,event_time_min,price_sum,price_mean,product_id_nunique,user_session_nunique,brand_nunique_cart,brand_nunique_purchase,brand_nunique_remove_from_cart,...,price_sum_remove_from_cart,price_sum_view,product_id_nunique_cart,product_id_nunique_purchase,product_id_nunique_remove_from_cart,product_id_nunique_view,user_session_nunique_cart,user_session_nunique_purchase,user_session_nunique_remove_from_cart,user_session_nunique_view
0,595412617,0,NaN,nan,nan,nan,nan,0,0,0,...,0.00000,45.11000,0,0,0,6,0,0,0,1
1,420652863,0,NaN,nan,nan,nan,nan,11,0,9,...,346.52000,1075.36000,50,0,45,152,10,0,7,14
2,593016733,0,NaN,nan,nan,nan,nan,0,0,0,...,0.00000,4.38000,0,0,0,1,0,0,0,1
3,595411904,0,2020-01-01 00:06:07 UTC,10.08000,5.04000,2.00000,1.00000,1,0,0,...,0.00000,51.16000,1,0,0,4,1,0,0,4
4,595411904,1,2020-01-01 00:06:07 UTC,10.08000,5.04000,2.00000,1.00000,1,0,0,...,0.00000,51.16000,1,0,0,4,1,0,0,4


In [32]:
def pipe_df_treino(df_input):
    df_events = pd.pivot_table(df_input, index=['user_id'], columns=['event_type'], values=[
    'event_time', 'product_id', 'category_id', 'brand', 'price', 'user_session'],
                           aggfunc={'user_session': 'nunique',
                                   'price': ['sum', 'mean'],
                                    'product_id': 'nunique',
                                    'category_id': 'nunique',
                                    'brand': 'nunique',
                                    'event_time': 'max'
                                   }, fill_value=0)
    df_events.columns = ['_'.join(col).strip() if col[1] != '' else col[0] for col in df_events.columns.values]
    df_train = df_target_final.merge(df_events, on='user_id', how='inner')
    
    return df_train

## Tratamento de dados

In [33]:
df_train.head()

,user_id,fl_purchase,event_time_min,price_sum,price_mean,product_id_nunique,user_session_nunique,brand_nunique_cart,brand_nunique_purchase,brand_nunique_remove_from_cart,...,price_sum_remove_from_cart,price_sum_view,product_id_nunique_cart,product_id_nunique_purchase,product_id_nunique_remove_from_cart,product_id_nunique_view,user_session_nunique_cart,user_session_nunique_purchase,user_session_nunique_remove_from_cart,user_session_nunique_view
0,595412617,0,NaN,nan,nan,nan,nan,0,0,0,...,0.00000,45.11000,0,0,0,6,0,0,0,1
1,420652863,0,NaN,nan,nan,nan,nan,11,0,9,...,346.52000,1075.36000,50,0,45,152,10,0,7,14
2,593016733,0,NaN,nan,nan,nan,nan,0,0,0,...,0.00000,4.38000,0,0,0,1,0,0,0,1
3,595411904,0,2020-01-01 00:06:07 UTC,10.08000,5.04000,2.00000,1.00000,1,0,0,...,0.00000,51.16000,1,0,0,4,1,0,0,4
4,595411904,1,2020-01-01 00:06:07 UTC,10.08000,5.04000,2.00000,1.00000,1,0,0,...,0.00000,51.16000,1,0,0,4,1,0,0,4


In [34]:
df_train.columns

Index(['user_id', 'fl_purchase', 'event_time_min', 'price_sum', 'price_mean',
       'product_id_nunique', 'user_session_nunique', 'brand_nunique_cart',
       'brand_nunique_purchase', 'brand_nunique_remove_from_cart',
       'brand_nunique_view', 'category_id_nunique_cart',
       'category_id_nunique_purchase', 'category_id_nunique_remove_from_cart',
       'category_id_nunique_view', 'event_time_max_cart',
       'event_time_max_purchase', 'event_time_max_remove_from_cart',
       'event_time_max_view', 'price_mean_cart', 'price_mean_purchase',
       'price_mean_remove_from_cart', 'price_mean_view', 'price_sum_cart',
       'price_sum_purchase', 'price_sum_remove_from_cart', 'price_sum_view',
       'product_id_nunique_cart', 'product_id_nunique_purchase',
       'product_id_nunique_remove_from_cart', 'product_id_nunique_view',
       'user_session_nunique_cart', 'user_session_nunique_purchase',
       'user_session_nunique_remove_from_cart', 'user_session_nunique_view'],
      dt

In [35]:
df_train.loc[df_train['event_time_max_cart']==0, 'event_time_max_cart'] = np.nan

In [36]:
df_train['event_time_max_cart'] = pd.to_datetime(df_train['event_time_max_cart']).astype(str)

In [37]:
colDatas = [ 'event_time_max_cart',
       'event_time_max_purchase', 'event_time_max_remove_from_cart',
       'event_time_max_view']

for col in colDatas:
    df_train.loc[df_train[col]==0, col] = np.nan
    df_train.loc[df_train[col]=='0', col] = np.nan
    df_train[col] = pd.to_datetime(df_train[col].astype(str))

In [38]:
def pipe_data_df_train(df_train):
    
    #tratamento de datas
    colDatas = [ 'event_time_max_cart',
       'event_time_max_purchase', 'event_time_max_remove_from_cart',
       'event_time_max_view']

    for col in colDatas:
        df_train.loc[df_train[col]==0, col] = np.nan
        df_train.loc[df_train[col]=='0', col] = np.nan
        df_train[col] = pd.to_datetime(df_train[col].astype(str))
    
    return df_train

In [39]:
#análise de outliers
df_train.describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

,user_id,fl_purchase,price_sum,price_mean,product_id_nunique,user_session_nunique,brand_nunique_cart,brand_nunique_purchase,brand_nunique_remove_from_cart,brand_nunique_view,...,price_sum_remove_from_cart,price_sum_view,product_id_nunique_cart,product_id_nunique_purchase,product_id_nunique_remove_from_cart,product_id_nunique_view,user_session_nunique_cart,user_session_nunique_purchase,user_session_nunique_remove_from_cart,user_session_nunique_view
count,64263.00000,64263.00000,19589.00000,19589.00000,19589.00000,19589.00000,64263.00000,64263.00000,64263.00000,64263.00000,...,64263.00000,64263.00000,64263.00000,64263.00000,64263.00000,64263.00000,64263.00000,64263.00000,64263.00000,64263.00000
mean,513937877.43359,0.15276,55.95029,6.50549,11.40267,1.25805,1.61264,0.62856,1.11125,2.59955,...,42.23742,149.35573,7.64462,2.21496,4.91351,9.92191,1.53116,0.26172,0.92800,4.92699
std,88011250.46391,0.35976,60.70328,8.93789,11.94035,0.71137,2.90899,1.68781,2.46441,3.92860,...,204.59246,436.62779,21.57239,7.08649,16.33828,24.59087,3.20880,0.64938,2.17710,27.42331
min,4661182.00000,0.00000,-12.83000,-1.06917,1.00000,1.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1%,231300887.44000,0.00000,5.00000,1.31453,1.00000,1.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
10%,394769404.20000,0.00000,12.38000,2.35790,2.00000,1.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,3.17000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,1.00000
25%,473730260.50000,0.00000,21.07000,3.22200,4.00000,1.00000,0.00000,0.00000,0.00000,1.00000,...,0.00000,9.52000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,1.00000
50%,545394854.00000,0.00000,40.86000,4.60800,8.00000,1.00000,0.00000,0.00000,0.00000,1.00000,...,0.00000,35.55000,1.00000,0.00000,0.00000,3.00000,1.00000,0.00000,0.00000,2.00000
75%,580861416.50000,0.00000,67.40000,6.74611,14.00000,1.00000,2.00000,0.00000,1.00000,3.00000,...,20.86000,124.77000,7.00000,0.00000,3.00000,9.00000,2.00000,0.00000,1.00000,4.00000
90%,590452794.60000,1.00000,111.93000,10.95250,23.00000,2.00000,5.00000,3.00000,4.00000,7.00000,...,107.21400,347.10000,22.00000,8.00000,14.00000,24.00000,4.00000,1.00000,3.00000,9.00000


In [40]:
colOutliers = ['brand_nunique_cart',
       'brand_nunique_purchase', 'brand_nunique_remove_from_cart',
       'brand_nunique_view', 'category_id_nunique_cart',
       'category_id_nunique_purchase', 'category_id_nunique_remove_from_cart',
       'category_id_nunique_view', 'price_mean_cart', 'price_mean_purchase',
       'price_mean_remove_from_cart', 'price_mean_view', 'price_sum_cart',
       'price_sum_purchase', 'price_sum_remove_from_cart', 'price_sum_view',
       'product_id_nunique_cart', 'product_id_nunique_purchase',
       'product_id_nunique_remove_from_cart', 'product_id_nunique_view',
       'user_session_nunique_cart', 'user_session_nunique_purchase',
       'user_session_nunique_remove_from_cart', 'user_session_nunique_view']

In [41]:
limite_inferior={}
limite_superior={}
for col in colOutliers:
    limite_inferior[col] = df_train[col].mean()-3*df_train[col].std()
    limite_superior[col] = df_train[col].mean()+3*df_train[col].std()
    
    df_train.loc[df_train[col]<limite_inferior[col], col] = limite_inferior[col]
    df_train.loc[df_train[col]>limite_superior[col], col] = limite_superior[col]

## PipeLine de remoção de outtliers

In [42]:
def treat_outliers_df_train(df_train):
    colOutliers = ['brand_nunique_cart',
       'brand_nunique_purchase', 'brand_nunique_remove_from_cart',
       'brand_nunique_view', 'category_id_nunique_cart',
       'category_id_nunique_purchase', 'category_id_nunique_remove_from_cart',
       'category_id_nunique_view', 'price_mean_cart', 'price_mean_purchase',
       'price_mean_remove_from_cart', 'price_mean_view', 'price_sum_cart',
       'price_sum_purchase', 'price_sum_remove_from_cart', 'price_sum_view',
       'product_id_nunique_cart', 'product_id_nunique_purchase',
       'product_id_nunique_remove_from_cart', 'product_id_nunique_view',
       'user_session_nunique_cart', 'user_session_nunique_purchase',
       'user_session_nunique_remove_from_cart', 'user_session_nunique_view']
    
    limite_inferior = {'brand_nunique_cart': -6.004833095979496,
         'brand_nunique_purchase': -3.5780004215439645,
         'brand_nunique_remove_from_cart': -5.068561746151607,
         'brand_nunique_view': -6.932017605182884,
         'category_id_nunique_cart': -11.07431667806755,
         'category_id_nunique_purchase': -6.738470730230486,
         'category_id_nunique_remove_from_cart': -9.077262367943455,
         'category_id_nunique_view': -11.14905462711328,
         'price_mean_cart': -13.132182417046746,
         'price_mean_purchase': -7.891299470323626,
         'price_mean_remove_from_cart': -10.3267796129172,
         'price_mean_view': -42.57340229647938,
         'price_sum_cart': -212.22751452370815,
         'price_sum_purchase': -62.038219482628655,
         'price_sum_remove_from_cart': -259.0818533880454,
         'price_sum_view': -626.9658791927441,
         'product_id_nunique_cart': -34.02754501503467,
         'product_id_nunique_purchase': -12.773661306431203,
         'product_id_nunique_remove_from_cart': -25.81027980381112,
         'product_id_nunique_view': -37.23700766488005,
         'user_session_nunique_cart': -8.095226316303972,
         'user_session_nunique_purchase': -1.6864252447882975,
         'user_session_nunique_remove_from_cart': -5.603291463095019,
         'user_session_nunique_view': -77.34294576757591}
    
    limite_superior = {'brand_nunique_cart': 9.062944595074143,
 'brand_nunique_purchase': 4.712825933212028,
 'brand_nunique_remove_from_cart': 7.118416693443537,
 'brand_nunique_view': 11.867631362088076,
 'category_id_nunique_cart': 16.70336282800412,
 'category_id_nunique_purchase': 8.932004364801834,
 'category_id_nunique_remove_from_cart': 12.783190468507204,
 'category_id_nunique_view': 19.021533098159402,
 'price_mean_cart': 19.527567865300316,
 'price_mean_purchase': 10.305236780823986,
 'price_mean_remove_from_cart': 14.685236799007132,
 'price_mean_view': 69.32434448723467,
 'price_sum_cart': 295.4428317451849,
 'price_sum_purchase': 80.87727456277364,
 'price_sum_remove_from_cart': 332.5511357525984,
 'price_sum_view': 889.4793573698139,
 'product_id_nunique_cart': 47.79611910132727,
 'product_id_nunique_purchase': 16.63572768339362,
 'product_id_nunique_remove_from_cart': 34.285876857354275,
 'product_id_nunique_view': 55.04904655499391,
 'user_session_nunique_cart': 11.157548336751198,
 'user_session_nunique_purchase': 2.2098679723298065,
 'user_session_nunique_remove_from_cart': 7.459289471280133,
 'user_session_nunique_view': 87.19692083876774}
    
    for col in colOutliers:
        limite_inferior[col] = df_train[col].mean()-3*df_train[col].std()
        limite_superior[col] = df_train[col].mean()+3*df_train[col].std()
    
        df_train.loc[df_train[col]<limite_inferior[col], col] = limite_inferior[col]
        df_train.loc[df_train[col]>limite_superior[col], col] = limite_superior[col]
    
    return df_train

## Pipeline em produção

In [43]:
df_input_teste = pd.read_csv(r'C:\DATASETS\day5\2020-Jan.csv')
df_target_teste = pd.read_csv(r'C:\DATASETS\day5\2020-Feb.csv')

In [44]:
df_fev = pipe_data_clean(df_target_teste)

df_fev = pipe_df_target(df_fev)

In [45]:
pipeline_Target = (df_target_teste.pipe(pipe_data_clean)
                  .pipe(pipe_data_clean)
                  .pipe(pipe_df_target)
                  )

In [46]:
pipeline_Target

,user_id,fl_purchase,event_time_min,price_sum,price_mean,product_id_nunique,user_session_nunique
0,485174092,0,2020-02-24 19:49:18 UTC,36.00000,3.00000,12.00000,1.00000
1,594621622,0,NaN,nan,nan,nan,nan
2,495404942,0,NaN,nan,nan,nan,nan
3,564814969,0,NaN,nan,nan,nan,nan
4,551205603,0,NaN,nan,nan,nan,nan
...,...,...,...,...,...,...,...
416665,622090098,0,NaN,nan,nan,nan,nan
416666,126992229,0,NaN,nan,nan,nan,nan
416667,622090052,0,NaN,nan,nan,nan,nan
416668,622090237,0,NaN,nan,nan,nan,nan
